### Pyspark GroupBy and Aggregate Functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab/ssafy_second_pjt/spark

/content/drive/MyDrive/Colab/ssafy_second_pjt/spark


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 47.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e26fd9b3ab6a4a95a13310cbcbae30162a0fa573febadb0f8a07119387a7be81
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tutorial_05').getOrCreate()

In [ ]:
sdf = spark.read.csv('test3.csv', header=True, inferSchema=True)
sdf.show()

+----+-----------+------+
|Name|Departments|Salary|
+----+-----------+------+
|   A|         DS| 10000|
|   B|         CS|  8000|
|   C|       STAT| 12000|
|   C|    BigData|  9000|
|   D|        IOT| 15000|
|   D|         CS| 18000|
|   E|       STAT| 20000|
|   E|         CS|  9000|
|   E|    BigData| 13000|
|   F|         DS| 17000|
+----+-----------+------+



In [ ]:
sdf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- Salary: integer (nullable = true)



### groupBy

In [ ]:
### groupBy 
sdf.groupBy('Name').sum().show()

+----+-----------+
|Name|sum(Salary)|
+----+-----------+
|   F|      17000|
|   E|      42000|
|   B|       8000|
|   D|      33000|
|   C|      21000|
|   A|      10000|
+----+-----------+



In [ ]:
### groupBy Departments
sdf.groupBy('Departments').sum().show()

+-----------+-----------+
|Departments|sum(Salary)|
+-----------+-----------+
|        IOT|      15000|
|    BigData|      22000|
|         CS|      35000|
|       STAT|      32000|
|         DS|      27000|
+-----------+-----------+



In [ ]:
sdf.groupBy('Departments').mean().show()

+-----------+------------------+
|Departments|       avg(Salary)|
+-----------+------------------+
|        IOT|           15000.0|
|    BigData|           11000.0|
|         CS|11666.666666666666|
|       STAT|           16000.0|
|         DS|           13500.0|
+-----------+------------------+



In [ ]:
sdf.groupBy('Departments').count().show()

+-----------+-----+
|Departments|count|
+-----------+-----+
|        IOT|    1|
|    BigData|    2|
|         CS|    3|
|       STAT|    2|
|         DS|    2|
+-----------+-----+



In [ ]:
sdf.groupBy('Departments').max().show()

+-----------+-----------+
|Departments|max(Salary)|
+-----------+-----------+
|        IOT|      15000|
|    BigData|      13000|
|         CS|      18000|
|       STAT|      20000|
|         DS|      17000|
+-----------+-----------+



### agg
- spark DF에 `groupBy()` method 적용하면, **GroupedData** 객체가 반환되고, 여기에 aggregation method (`agg()` )적용 가능
- spark DataFrame에 aggregation 메소드를 적용 시에는 `pyspark.sql.functions` 모듈의 `max, min, sum` 등의 함수를 이용해야함.
    - 바로 aggregation은 `count()` 만 가능함
    - 그 외에는 `select()` 와 같이 호출해야 함
    - **select() 는 전부 DF로 반환된다고 생각!**

+ 문자열, col()  둘 다 가능!
+ alias()는 column 단위로 적용됨!

In [ ]:
sdf.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|     131000|
+-----------+



In [ ]:
from pyspark.sql.functions import col, max, min, avg, sum, count
sdf.groupBy('Name').agg(max('Salary').alias('max salary'), min('Salary'), avg('Salary'), count('Salary')).show()

+----+----------+-----------+-----------+-------------+
|Name|max salary|min(Salary)|avg(Salary)|count(Salary)|
+----+----------+-----------+-----------+-------------+
|   F|     17000|      17000|    17000.0|            1|
|   E|     20000|       9000|    14000.0|            3|
|   B|      8000|       8000|     8000.0|            1|
|   D|     18000|      15000|    16500.0|            2|
|   C|     12000|       9000|    10500.0|            2|
|   A|     10000|      10000|    10000.0|            1|
+----+----------+-----------+-----------+-------------+

